In [2]:
import os
import pandas as pd
from nltk.corpus import stopwords
from pythainlp.corpus.common import thai_stopwords

file_name = 'merged_data.xlsx'
synonyms_file = 'synonyms_NHSO.xlsx'
en_stop = set(stopwords.words('english'))
th_stop = set(thai_stopwords())

In [10]:
df = pd.read_excel('merged_data.xlsx')
df

,ประเด็น,คำถาม,คำตอบ
0,สิทธิไม่ตรงตามจริง ติดสิทธิข้าราชการ,บุตรข้าราชการ อายุ 18 ปี แต่งงาน จดทะเบียนสมรส...,ไม่สามารถใช้สิทธิเบิกข้าราชการได้ เนื่องจากบรร...
1,สิทธิไม่ตรงตามจริง ติดสิทธิข้าราชการ,บุตรข้าราชการบรรลุนิติภาวะ ไม่สามารถใช้สิทธิเบ...,1.บุตรอายุครบ 20 ปีบริบูรณ์\n2.บุตรแต่งงาน จดท...
2,สิทธิไม่ตรงตามจริง ติดสิทธิข้าราชการ,ข้าราชการเกษียณอายุได้รับบำเหน็จ แต่ไม่สามารถห...,หนังสือรับรองที่ออกโดยกำนันหรือผู้ใหญ่บ้านหรือ...
3,สิทธิไม่ตรงตามจริง ติดสิทธิข้าราชการ,ข้าราชการกรมบัญชีกลาง หมดสิทธิแล้ว ต้องการปรับ...,1.ติดต่อนายทะเบียนต้นสังกัดเพื่อส่งข้อมูลการหม...
4,สิทธิไม่ตรงตามจริง ติดสิทธิข้าราชการ,พนักงานองค์กรปกครองส่วนท้องถิ่น (อปท.) หมดสิทธ...,1330 ไม่สามารถปรับปรุงฐานสิทธิขององค์กรปกครองส...
...,...,...,...
277,บริการฟื้นฟูสมรรถภาพด้านการแพทย์,ผู้ป่วยมีสิทธิบัตรทอง เป็นคนพิการทางการได้ยิน ...,หากมีสิทธิบัตรทองประเภทสิทธิย่อยคนพิการ สามารถ...
278,บริการฟื้นฟูสมรรถภาพด้านการแพทย์,เป็นผู้ป่วยอัมพาต ติดเตียง ต้องการขอรับที่นอน...,ได้ ตามข้อบ่งชี้ทางการแพทย์
279,บริการฟื้นฟูสมรรถภาพด้านการแพทย์,โปรแกรมไว้สำหรับบันทึกข้อมูลเบิกอุปกรณ์คนพิการ...,ไปที่หน้าเว็บหลัก สปสช. www.nhso.go.th บริกา...
280,บริการฟื้นฟูสมรรถภาพด้านการแพทย์,กรณีผู้ป่วยสิทธิว่าง เป็นผู้พิการ โดยยังไม่ได้...,ให้หน่วยบริการทำการลงทะเบียน และปรับประเภทสิทธ...


In [3]:
def clean(text):
    text = text.translate(str.maketrans('', '', '''!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~'''))
    text = text.lower()
    text = text.replace('\n', '')
    text = text.replace('\t', '')
    text = text.replace('\r', '')
    return text

def clean_q(text):
    text = text.replace('\n', '')
    text = text.replace('\t', '')
    text = text.replace('\r', '')
    return text

def load_synonyms(synonyms_file):
    if os.path.exists(synonyms_file):
        df_synonyms = pd.read_excel(synonyms_file)
        synonym_dict = {}
        for index, row in df_synonyms.iterrows():
            main_word = row['main_word']
            synonyms = row['synonyms'].split(',')  # Assuming synonyms are comma-separated
            synonym_dict[main_word] = synonyms
        return synonym_dict
    return {}

def expand_patterns_with_synonyms(patterns, synonym_dict):
    expanded_patterns = set(patterns)
    for pattern in patterns:
        if pattern in synonym_dict:
            expanded_patterns.update(synonym_dict[pattern])
    return list(expanded_patterns)

def search(patterns, data, synonym_dict):
    expanded_patterns = expand_patterns_with_synonyms(patterns, synonym_dict)
    series = pd.Series(data)
    counts_df = pd.DataFrame(index=series.index)
    nvocab = len(expanded_patterns)
    for pattern in expanded_patterns:
        counts_df[pattern] = series.str.count(pattern)
    logit = counts_df > 0
    counts_df = counts_df / logit.sum(axis=0)
    counts_df['tier'] = logit.sum(axis=1)
    counts_df['total_words'] = counts_df.iloc[:, :-1].sum(axis=1)
    counts_df['logit'] = counts_df['tier'] == nvocab
    return counts_df.sort_values(by=['tier', 'total_words'], ascending=[False, False])

def upload_and_merge_excel(file_name):
    if os.path.exists(file_name):
        existing_data = pd.read_excel(file_name)
    else:
        existing_data = pd.DataFrame()

    uploaded_file = input("Enter the path to the Excel file to upload: ")
    if os.path.exists(uploaded_file):
        df_new = pd.read_excel(uploaded_file)
        df_combined = pd.concat([existing_data, df_new], ignore_index=True)
        return df_combined
    return existing_data

In [4]:
def view_excel(file_name):
    if os.path.exists(file_name):
        df = pd.read_excel(file_name)
        print(df)
    else:
        print(f"File {file_name} not found")

def edit_excel(file_name):
    existing_data = upload_and_merge_excel(file_name)
    print("Current Data:")
    print(existing_data)

    action = input("Enter 'add' to add a row, 'delete' to delete a row, or 'done' to finish: ")

    while action.lower() != 'done':
        if action.lower() == 'add':
            new_row = input("Enter the new row data as comma-separated values: ")
            new_row_data = new_row.split(',')
            new_row_df = pd.DataFrame([new_row_data], columns=existing_data.columns)
            existing_data = pd.concat([existing_data, new_row_df], ignore_index=True)
            print("Row added. Current Data:")
            print(existing_data)
        
        elif action.lower() == 'delete':
            row_to_delete = int(input("Enter the index of the row to delete: "))
            if row_to_delete in existing_data.index:
                existing_data = existing_data.drop(row_to_delete).reset_index(drop=True)
                print("Row deleted. Current Data:")
                print(existing_data)
            else:
                print(f"Row index {row_to_delete} not found.")
        
        action = input("Enter 'add' to add a row, 'delete' to delete a row, or 'done' to finish: ")

    return existing_data

In [20]:
# Load synonyms data
synonym_dict = load_synonyms(synonyms_file)

# Extract patterns from main_word column of synonyms file
patterns = list(synonym_dict.keys())

# Example usage of the search function
# Replace 'data' with your actual data source
data = ["ไต บัตรทอง"]
result = search(patterns, data, synonym_dict)
print(result)

   tier  total_words  logit
0   0.0          0.0   True


In [22]:
s_index = result[result['logit']==True].index.to_list()
if s_index:
    for i in s_index:
        print('Question : ', clean_q(df['คำถาม'][i]))
        print('Answer :\n', df['คำตอบ'][i].strip())
        print('')
else:
    print('Not found')

Question :  บุตรข้าราชการ อายุ 18 ปี แต่งงาน จดทะเบียนสมรส ยังสามารถใช้สิทธิเบิกข้าราชการได้หรือไม่
Answer :
 ไม่สามารถใช้สิทธิเบิกข้าราชการได้ เนื่องจากบรรลุนิติภาวะจากการจดทะเบียนสมรส ต้องใช้สิทธิบัตรทอง



In [19]:
top_5_logit = result.nlargest(5, 'logit')
print("Top 5 rows with the highest 'logit' values:")
print(top_5_logit)

Top 5 rows with the highest 'logit' values:
   tier  total_words  logit
0   0.0          0.0   True
